In [27]:
import pandas as pd
%load_ext autoreload
%autoreload 2
    
import cga_utils
import prompt_versions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
dataset_tr = pd.read_json('dataset_raw/tatqa_dataset_train.json')    
dataset_t = pd.read_json('dataset_raw/tatqa_dataset_test.json')    
dataset_tg = pd.read_json('dataset_raw/tatqa_dataset_test_gold.json')    
dataset_d = pd.read_json('dataset_raw/tatqa_dataset_dev.json')    

sample_df=dataset_tg
#sample_df = dataset_tg.sample(frac=0.20, random_state=42)
#sample_df = dataset_tr.sample(frac=0.031, random_state=42)


In [52]:
from langchain_core.callbacks import BaseCallbackHandler
from langchain_community.chat_models import ChatOllama

class OllamaStatsHandler(BaseCallbackHandler):
    def __init__(self):
        self.reset()

    def reset(self):
        self.total_calls = 0
        self.total_eval_ns = 0
        self.total_total_ns = 0

    def on_llm_end(self, response, **kwargs):
        self.total_calls += 1

        metadata = response.generations[0][0].message.response_metadata
        
        self.total_eval_ns += metadata.get("eval_duration", 0)
        self.total_total_ns += metadata.get("total_duration", 0)

    def summary(self):
        return {
            "calls": self.total_calls,
            "eval_seconds": self.total_eval_ns / 1e9,
            "total_seconds": self.total_total_ns / 1e9,
        }





In [54]:
stats = OllamaStatsHandler()
llm = ChatOllama(model="qwen3:4b", temperature = 0.0, top_p = 1, repeat_penalty=1, presence_penalty=0, frequency_penalty=0, num_predict=400, num_ctx=4096, callbacks=[stats])  # vagy bármely más elérhető Ollama model
messages = prompt_versions.prompt_versions['V18'] #Base
messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 
import time

start = time.time()
resq3 = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df.iloc[1:10])
end = time.time()
end-start, (end-start)/len(resq3)

1771947945.9919074


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1166356.0 good answer:  995684.5  1.369s
<< (948,578+1,042,791)/2 || [1112987.0, 1219725.0, 1166356.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.75 good answer:  -9.03  1.284s
<< ((948,578-1,042,791)/1,042,791) * 100 || [1112987.0, 1219725.0, -8.750988952427802] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? Success: 643.0 1.076s
<< 17,845-17,202 || [17845.0, 17202.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 1.402s
<< 1,320 - 1,086 || [1086.0, 1320.0] || [1320.0, 1086.0] || True >>
d9b81db920283c307bd37d6000b

(29.323021411895752, 1.4661510705947876)

In [57]:
stats.summary()['total_seconds']/len(resq3)

1.45679909665

In [58]:
stats = OllamaStatsHandler()
llm = ChatOllama(model="gemma3:4b", temperature = 0.0, top_p = 1, repeat_penalty=1, presence_penalty=0, frequency_penalty=0, num_predict=400, num_ctx=4096, callbacks=[stats])  # vagy bármely más elérhető Ollama model
messages = prompt_versions.prompt_versions['V18'] #Base
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 
import time

start = time.time()
res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df.iloc[1:10])
end = time.time()
end-start, (end-start)/len(res)

1771948095.231846


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 5057548.5 good answer:  995684.5  4.732s
<< (948,578+1,042,791)/2 || [4820218.0, 5294879.0, 5057548.5] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.96 good answer:  -9.03  2.813s
<< ((948,578-1,042,791)/1,042,791) * 100 || [5294879.0, 4820218.0, -0.08964529689913595] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? failure: 643.0 good answer:  643  3.153s
<< 17,845-17,202 || [17845.0, 17202.0, 643.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 2.488s
<< 1,320 - 1,086 || [1320.0, 1086.0, 234.0] || [1320.0, 1086.0] || 

(52.56449794769287, 2.6282248973846434)

In [59]:
stats.summary()['total_seconds']/len(res)

2.6194378699

In [61]:
stats = OllamaStatsHandler()
llm = ChatOllama(model="gemma3n:e4b", temperature = 0.0, top_p = 1, repeat_penalty=1, presence_penalty=0, frequency_penalty=0, num_predict=400, num_ctx=4096, callbacks=[stats])  # vagy bármely más elérhető Ollama model
messages = prompt_versions.prompt_versions['V18'] #Base
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 
import time

start = time.time()
resn = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df.iloc[1:10])
end = time.time()
end-start, (end-start)/len(resn)

1771948234.2563686


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1082760.0 good answer:  995684.5  5.999s
<< (948,578+1,042,791)/2 || [1122729.0, 1042791.0, 1082760.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: 7.67 good answer:  -9.03  4.224s
<< ((948,578-1,042,791)/1,042,791) * 100 || [1042791.0, 1122729.0, 7.665773870315336] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? Success: 643.0 4.059s
<< 17,845-17,202 || [17845.0, 17202.0, 643.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 3.483s
<< 1,320 - 1,086 || [1320.0, 1086.0, 234.0] || [1320.0, 1086.0] || True >>
d9b81db920283c3

(92.30435180664062, 4.615217590332032)

In [62]:
stats.summary()['total_seconds']/len(resn)

4.60662578145

In [ ]:
llm = ChatOllama(model="qwen3:4b", temperature = 0.0, top_p = 1, repeat_penalty=1, presence_penalty=0, frequency_penalty=0, num_predict=400, num_ctx=4096)  # vagy bármely más elérhető Ollama model
messages = prompt_versions.prompt_versions['V18'].copy() #Base
messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 


res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df.iloc[1:10])


1771946415.5393987


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1166356.0 good answer:  995684.5  1.771s
<< (948,578+1,042,791)/2 || [1112987.0, 1219725.0, 1166356.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.75 good answer:  -9.03  1.291s
<< ((948,578-1,042,791)/1,042,791) * 100 || [1112987.0, 1219725.0, -8.750988952427802] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? Success: 643.0 1.072s
<< 17,845-17,202 || [17845.0, 17202.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 1.410s
<< 1,320 - 1,086 || [1086.0, 1320.0] || [1320.0, 1086.0] || True >>
d9b81db920283c307bd37d6000b

In [32]:
llm = ChatOllama(model="gemma3:4b", temperature = 0.0, top_p = 1, repeat_penalty=1, presence_penalty=0, frequency_penalty=0, num_predict=400, num_ctx=4096)  # vagy bármely más elérhető Ollama model
messages = prompt_versions.prompt_versions['V18'] #Base
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df.iloc[1:10])


1771946005.126998


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 5057548.5 good answer:  995684.5  2.848s
<< (948,578+1,042,791)/2 || [4820218.0, 5294879.0, 5057548.5] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.96 good answer:  -9.03  2.781s
<< ((948,578-1,042,791)/1,042,791) * 100 || [5294879.0, 4820218.0, -0.08964529689913595] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? failure: 643.0 good answer:  643  3.142s
<< 17,845-17,202 || [17845.0, 17202.0, 643.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 2.478s
<< 1,320 - 1,086 || [1320.0, 1086.0, 234.0] || [1320.0, 1086.0] || 

In [33]:
llm = ChatOllama(model="gemma3n:e4b", temperature = 0.0, top_p = 1, repeat_penalty=1, presence_penalty=0, frequency_penalty=0, num_predict=400, num_ctx=4096)  # vagy bármely más elérhető Ollama model
messages = prompt_versions.prompt_versions['V18'] #Base
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df.iloc[1:10])

1771946055.9024274


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1082760.0 good answer:  995684.5  5.951s
<< (948,578+1,042,791)/2 || [1122729.0, 1042791.0, 1082760.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: 7.67 good answer:  -9.03  4.239s
<< ((948,578-1,042,791)/1,042,791) * 100 || [1042791.0, 1122729.0, 7.665773870315336] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? Success: 643.0 4.053s
<< 17,845-17,202 || [17845.0, 17202.0, 643.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 3.476s
<< 1,320 - 1,086 || [1320.0, 1086.0, 234.0] || [1320.0, 1086.0] || True >>
d9b81db920283c3